In [2]:
import gym
from gym import spaces
import numpy as np

In [6]:
class ChargingEnvironment(gym.Env):
    def __init__(self, charging_capacity=100, charging_rate=10):
        super(ChargingEnvironment, self).__init__()
        
        #Environment parameters
        self.charging_capacity = charging_capacity # Total battery capacity
        self.charging_rate = charging_rate #charging rate in unites per hour
        
        #Observation spcae
        low = np.array([0.0, 0, 0], dtype = np.float32)
        high = np.array([1.0, 24, np.inf], dtype = np.float32)
        self.observation_space = spaces.Box(low = low, high = high, dtype = np.float32)
        
        #Action_space (charging decision)
        self.action_space = spaces.Discrete(2)  # 0: Do not charge, 1: Charge
        
        #Internal state variables
        self.current_charge = self.charging_capacity
        self.hours_til_next_shift = 24
        self.current_hour = 0
        
    def step(self, action):
        #update the environment based on the agents action
        if action == 1: #Charge
            self.current_charge += self.charging_rate
            self.current_charge = min(self.current_charge, self.charging_capacity)
            
        #Simulate time progression
        self.current_hour += 1
        self.hours_til_next_shift -= 1
        
        #Calculate reward(negative of charging cost)
        electricity_price = self.get_electricity_price(self.current_hour)
        reward = -action * self.charging_rate * electricity_price
        
        #Check for termination condition
        done = self.hours_til_next_shift = 0 or self.current_charge >= self.charging_capacity
        
        observation = self.get_observation()
        
        return observation, reward, done, {}
    
    def reset(self):
        #Reset the environment to its initial state
        self.current_charge = self.charging_capacity
        self.hours_til_next_shift = 24
        self.current_hour = 0
        
    def get_observation(self):
        return np.array([
            self.current_charge / self.charging_capacity, #normalized charging level
            self.hours_til_next_shift,
            self.get_electricity_price(self.current_hour)
            
        ])
        
    def get_electricity_price(self, hour):
    # This is a simple example where the price is higher during the day and lower at night
        if 8 <= hour <= 18:
            return 0.15  # Daytime electricity price
        else:
            return 0.08  # Nighttime electricity price
        

In [ ]:
from pettingzoo import ParallelEnv

class CustomEnvironment(ParallelEnv):
    metadata = {
        "name": "custom_environment_v0",
    }
    
    def __init__(self):
        pass
    
    def reset(self, seed=None, options = None):
        pass
    
    def step(self, actions):
        pass
    
    def render(self):
        pass
    
    def observation_space(self, agent):
        return self.observation_spaces[agent]
    
    def action_space(self, agent):
        return self.action_spaces[agent]

In [1]:
import numpy as np 
import random
import time
import gymnasium as gym